In [ ]:
import numpy as np
import pandas as pd

from OptimalHedging.GBM import GBMSimulator
from OptimalHedging.Heston import HestonSimulator
from OptimalHedging.JumpDiff import JumpDiffusionSimulator

np.random.seed(123)

M     = 1000
N     = 252
S0    = 10.0
K0    = 15.0
T     = 1.0
t0    = 0.0

mu    = 0.05
sigma = 0.2

kappa   = 1.0
theta   = 0.1
sigma_v = 0.1
corr    = 0.7

lam   = 0.5
meanJ = -0.03
stdJ  = 0.12


gamma = 1.0
a     = 1.0
k     = 2.0
beta  = 0.95
U     = 10.0

risk_specs = {
    "ele" : {"a": a},
    "elw" : {"k": k, "U": U},
    "entl": {"gamma": gamma, "U": U},
    "ente": {"gamma": gamma, "a": a, "U": U},
    "entw": {"gamma": gamma, "k": k, "U": U},
    "esl" : {"beta": beta},
}


In [ ]:
# ============================================================
# BLOCO 2 - SURFACE: MR(t, moneyness) for GBM, Heston, Jump
# Saves CSV after EACH (model, risk_type)
# ============================================================
n_t = 21
n_k = 21
t_grid = np.linspace(t0, T, n_t)
K_grid = np.linspace(8.0, 16.0, n_k)

max_iter = 20
tol = 1e-4
alpha = 1e-3
verbose = False

# -----------------------------
# GBM
# -----------------------------
sim = GBMSimulator(M=M, N=N, S0=S0, K=K0, T=T, t0=t0, mu=mu, sigma=sigma)
sim.simulate_S()  # fixa os paths para toda a superfície

for risk_type, risk_kwargs in risk_specs.items():
    rows = []
    total = len(K_grid) * len(t_grid)
    done = 0
    print(f"[SURFACE] model=GBM | risk={risk_type} | total={total}")

    for K in K_grid:
        sim.K = float(K)
        m = float(S0 / sim.K)

        sim.simulate_H()  # payoff depende de K

        for t in t_grid:
            MR, _ = sim.compute_MR(
                t_idx=float(t),
                risk_type=risk_type,
                risk_kwargs=risk_kwargs,
                max_iter=max_iter,
                tol=tol,
                alpha=alpha,
                verbose=verbose
                # NÃO forçar kind="Delta" aqui
            )
            rows.append({"model": "GBM", "risk_type": risk_type, "t": float(t),
                         "K": float(K), "moneyness": m, "MR": float(MR)})
            done += 1
            if done == 1 or done % max(1, total // 20) == 0 or done == total:
                print(f"[SURFACE] model=GBM | risk={risk_type} | {100*done/total:6.2f}%")

    df = pd.DataFrame(rows)
    df.to_csv(f"surface_GBM_{risk_type}.csv", index=False)
    print(f"[SURFACE] saved surface_GBM_{risk_type}.csv | rows={len(df)}")

# -----------------------------
# Heston
# -----------------------------
sim = HestonSimulator(
    M=M, N=N, S0=S0, K=K0, T=T, t0=t0,
    mu=mu, sigma=sigma,
    kappa=kappa, theta=theta, sigma_v=sigma_v, corr=corr
)
sim.simulate_S()

for risk_type, risk_kwargs in risk_specs.items():
    rows = []
    total = len(K_grid) * len(t_grid)
    done = 0
    print(f"[SURFACE] model=Heston | risk={risk_type} | total={total}")

    for K in K_grid:
        sim.K = float(K)
        m = float(S0 / sim.K)

        sim.simulate_H()

        for t in t_grid:
            MR, _ = sim.compute_MR(
                t_idx=float(t),
                risk_type=risk_type,
                risk_kwargs=risk_kwargs,
                max_iter=max_iter,
                tol=tol,
                alpha=alpha,
                verbose=verbose
                # idem: não forçar kind aqui
            )
            rows.append({"model": "Heston", "risk_type": risk_type, "t": float(t),
                         "K": float(K), "moneyness": m, "MR": float(MR)})
            done += 1
            if done == 1 or done % max(1, total // 20) == 0 or done == total:
                print(f"[SURFACE] model=Heston | risk={risk_type} | {100*done/total:6.2f}%")

    df = pd.DataFrame(rows)
    df.to_csv(f"surface_Heston_{risk_type}.csv", index=False)
    print(f"[SURFACE] saved surface_Heston_{risk_type}.csv | rows={len(df)}")

# -----------------------------
# Jump
# -----------------------------
sim = JumpDiffusionSimulator(
    M=M, N=N, S0=S0, K=K0, T=T, t0=t0,
    mu=mu, sigma=sigma,
    lam=lam, meanJ=meanJ, stdJ=stdJ
)
sim.simulate_S()

for risk_type, risk_kwargs in risk_specs.items():
    rows = []
    total = len(K_grid) * len(t_grid)
    done = 0
    print(f"[SURFACE] model=Jump | risk={risk_type} | total={total}")

    for K in K_grid:
        sim.K = float(K)
        m = float(S0 / sim.K)

        sim.simulate_H()

        for t in t_grid:
            MR, _ = sim.compute_MR(
                t_idx=float(t),
                risk_type=risk_type,
                risk_kwargs=risk_kwargs,
                max_iter=max_iter,
                tol=tol,
                alpha=alpha,
                verbose=verbose
            )
            rows.append({"model": "Jump", "risk_type": risk_type, "t": float(t),
                         "K": float(K), "moneyness": m, "MR": float(MR)})
            done += 1
            if done == 1 or done % max(1, total // 20) == 0 or done == total:
                print(f"[SURFACE] model=Jump | risk={risk_type} | {100*done/total:6.2f}%")

    df = pd.DataFrame(rows)
    df.to_csv(f"surface_Jump_{risk_type}.csv", index=False)
    print(f"[SURFACE] saved surface_Jump_{risk_type}.csv | rows={len(df)}")


In [ ]:
# ============================================================
# BLOCO 3 - PROCESS SENS (ATM): DeltaMR on (param1,param2), t_idx in {0.2,0.8}
# Saves CSV after EACH (model, risk_type, t_idx)
# ============================================================
t_list = [0.2, 0.8]
max_iter = 20
tol = 1e-4
alpha = 1e-3
verbose = False

# grids
mu_grid    = np.linspace(0.025, 0.075, 9)
sig_grid   = np.linspace(0.10, 0.40, 9)

kappa_grid = np.linspace(1.0, 4.0, 9)
theta_grid = np.linspace(0.1, 0.3, 9)

meanJ_grid = np.linspace(-0.3, 0.3, 9)
lam_grid   = np.linspace(0.0, 1.0, 9)

# -----------------------------
# GBM: mu x sigma
# -----------------------------
sim = GBMSimulator(M=M, N=N, S0=S0, K=S0, T=T, t0=t0, mu=mu, sigma=sigma)

for risk_type, risk_kwargs in risk_specs.items():
    for t_idx in t_list:
        # baseline (standard mu,sigma)
        sim.mu = float(mu)
        sim.sigma = float(sigma)
        sim.simulate_S()
        sim.simulate_H()
        MR0, _ = sim.compute_MR(
            t_idx=float(t_idx),
            risk_type=risk_type,
            risk_kwargs=risk_kwargs,
            max_iter=max_iter,
            tol=tol,
            alpha=alpha,
            verbose=verbose
        )

        rows = []
        total = len(mu_grid) * len(sig_grid)
        done = 0
        print(f"[SENS] model=GBM | risk={risk_type} | t_idx={t_idx} | total={total}")

        for mu_i in mu_grid:
            for sig_i in sig_grid:
                sim.mu = float(mu_i)
                sim.sigma = float(sig_i)
                sim.simulate_S()
                sim.simulate_H()

                MR, _ = sim.compute_MR(
                    t_idx=float(t_idx),
                    risk_type=risk_type,
                    risk_kwargs=risk_kwargs,
                    max_iter=max_iter,
                    tol=tol,
                    alpha=alpha,
                    verbose=verbose
                )

                rows.append({
                    "model": "GBM", "risk_type": risk_type, "t_idx": float(t_idx),
                    "mu": float(mu_i), "sigma": float(sig_i),
                    "MR": float(MR), "MR0": float(MR0), "DeltaMR": float(MR - MR0)
                })

                done += 1
                if done == 1 or done % max(1, total // 20) == 0 or done == total:
                    print(f"[SENS] model=GBM | risk={risk_type} | t_idx={t_idx} | {100*done/total:6.2f}%")

        df = pd.DataFrame(rows)
        df.to_csv(f"sens_process_GBM_{risk_type}_t{t_idx}.csv", index=False)
        print(f"[SENS] saved sens_process_GBM_{risk_type}_t{t_idx}.csv | rows={len(df)}")

# -----------------------------
# Heston: kappa x theta
# -----------------------------
sim = HestonSimulator(
    M=M, N=N, S0=S0, K=S0, T=T, t0=t0,
    mu=mu, sigma=sigma,
    kappa=kappa, theta=theta, sigma_v=sigma_v, corr=corr
)

for risk_type, risk_kwargs in risk_specs.items():
    for t_idx in t_list:
        sim.kappa = float(kappa)
        sim.theta = float(theta)
        sim.simulate_S()
        sim.simulate_H()
        MR0, _ = sim.compute_MR(
            t_idx=float(t_idx),
            risk_type=risk_type,
            risk_kwargs=risk_kwargs,
            max_iter=max_iter,
            tol=tol,
            alpha=alpha,
            verbose=verbose
        )

        rows = []
        total = len(kappa_grid) * len(theta_grid)
        done = 0
        print(f"[SENS] model=Heston | risk={risk_type} | t_idx={t_idx} | total={total}")

        for k_i in kappa_grid:
            for th_i in theta_grid:
                sim.kappa = float(k_i)
                sim.theta = float(th_i)
                sim.simulate_S()
                sim.simulate_H()

                MR, _ = sim.compute_MR(
                    t_idx=float(t_idx),
                    risk_type=risk_type,
                    risk_kwargs=risk_kwargs,
                    max_iter=max_iter,
                    tol=tol,
                    alpha=alpha,
                    verbose=verbose
                )

                rows.append({
                    "model": "Heston", "risk_type": risk_type, "t_idx": float(t_idx),
                    "kappa": float(k_i), "theta": float(th_i),
                    "MR": float(MR), "MR0": float(MR0), "DeltaMR": float(MR - MR0)
                })

                done += 1
                if done == 1 or done % max(1, total // 20) == 0 or done == total:
                    print(f"[SENS] model=Heston | risk={risk_type} | t_idx={t_idx} | {100*done/total:6.2f}%")

        df = pd.DataFrame(rows)
        df.to_csv(f"sens_process_Heston_{risk_type}_t{t_idx}.csv", index=False)
        print(f"[SENS] saved sens_process_Heston_{risk_type}_t{t_idx}.csv | rows={len(df)}")

# -----------------------------
# Jump: meanJ x lam
# -----------------------------
sim = JumpDiffusionSimulator(
    M=M, N=N, S0=S0, K=S0, T=T, t0=t0,
    mu=mu, sigma=sigma,
    lam=lam, meanJ=meanJ, stdJ=stdJ
)

for risk_type, risk_kwargs in risk_specs.items():
    for t_idx in t_list:
        sim.meanJ = float(meanJ)
        sim.lam = float(lam)
        sim.simulate_S()
        sim.simulate_H()
        MR0, _ = sim.compute_MR(
            t_idx=float(t_idx),
            risk_type=risk_type,
            risk_kwargs=risk_kwargs,
            max_iter=max_iter,
            tol=tol,
            alpha=alpha,
            verbose=verbose
        )

        rows = []
        total = len(meanJ_grid) * len(lam_grid)
        done = 0
        print(f"[SENS] model=Jump | risk={risk_type} | t_idx={t_idx} | total={total}")

        for mJ in meanJ_grid:
            for l in lam_grid:
                sim.meanJ = float(mJ)
                sim.lam = float(l)
                sim.simulate_S()
                sim.simulate_H()

                MR, _ = sim.compute_MR(
                    t_idx=float(t_idx),
                    risk_type=risk_type,
                    risk_kwargs=risk_kwargs,
                    max_iter=max_iter,
                    tol=tol,
                    alpha=alpha,
                    verbose=verbose
                )

                rows.append({
                    "model": "Jump", "risk_type": risk_type, "t_idx": float(t_idx),
                    "meanJ": float(mJ), "lam": float(l),
                    "MR": float(MR), "MR0": float(MR0), "DeltaMR": float(MR - MR0)
                })

                done += 1
                if done == 1 or done % max(1, total // 20) == 0 or done == total:
                    print(f"[SENS] model=Jump | risk={risk_type} | t_idx={t_idx} | {100*done/total:6.2f}%")

        df = pd.DataFrame(rows)
        df.to_csv(f"sens_process_Jump_{risk_type}_t{t_idx}.csv", index=False)
        print(f"[SENS] saved sens_process_Jump_{risk_type}_t{t_idx}.csv | rows={len(df)}")



In [ ]:
# ============================================================
# BLOCO 4 - AVERSION SENS (ATM): DeltaMR vs param, t_idx in {0.2,0.8}
# Saves CSV after EACH (model, risk_type, param, t_idx)
# ============================================================

t_list = [0.2, 0.8]
max_iter = 20
tol = 1e-4
alpha = 1e-3
verbose = False

gamma_grid = np.linspace(0.1, 2.0, 15)
a_grid     = np.linspace(0.5, 2.0, 15)
k_grid     = np.linspace(1.1, 3.0, 15)
beta_list  = [0.9, 0.95, 0.975, 0.99]

# ============================================================
# GBM (kind="Delta")
# ============================================================

sim = GBMSimulator(M=M, N=N, S0=S0, K=S0, T=T, t0=t0, mu=mu, sigma=sigma)
sim.simulate_S()
sim.simulate_H()

# gamma: entl, ente, entw
for risk_type in ["entl", "ente", "entw"]:
    base_kwargs = dict(risk_specs[risk_type])
    for t_idx in t_list:
        MR0, _ = sim.compute_MR(
            t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=base_kwargs,
            kind="Delta", max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose
        )

        rows = []
        total = len(gamma_grid)
        done = 0
        print(f"[AVERS] model=GBM | risk={risk_type} | param=gamma | t_idx={t_idx} | total={total}")

        for g in gamma_grid:
            rk = dict(base_kwargs)
            rk["gamma"] = float(g)

            MR, _ = sim.compute_MR(
                t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=rk,
                kind="Delta", max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose
            )

            rows.append({
                "model": "GBM", "risk_type": risk_type, "t_idx": float(t_idx),
                "param_name": "gamma", "param_value": float(g),
                "MR": float(MR), "MR0": float(MR0), "DeltaMR": float(MR - MR0)
            })

            done += 1
            if done == 1 or done % max(1, total // 10) == 0 or done == total:
                print(f"[AVERS] model=GBM | risk={risk_type} | gamma | t_idx={t_idx} | {100*done/total:6.2f}%")

        df = pd.DataFrame(rows)
        df.to_csv(f"sens_aversion_GBM_{risk_type}_gamma_t{t_idx}.csv", index=False)

# a: ele, ente
for risk_type in ["ele", "ente"]:
    base_kwargs = dict(risk_specs[risk_type])
    for t_idx in t_list:
        MR0, _ = sim.compute_MR(
            t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=base_kwargs,
            kind="Delta", max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose
        )

        rows = []
        total = len(a_grid)
        done = 0
        print(f"[AVERS] model=GBM | risk={risk_type} | param=a | t_idx={t_idx} | total={total}")

        for av in a_grid:
            rk = dict(base_kwargs)
            rk["a"] = float(av)

            MR, _ = sim.compute_MR(
                t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=rk,
                kind="Delta", max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose
            )

            rows.append({
                "model": "GBM", "risk_type": risk_type, "t_idx": float(t_idx),
                "param_name": "a", "param_value": float(av),
                "MR": float(MR), "MR0": float(MR0), "DeltaMR": float(MR - MR0)
            })

            done += 1
            if done == 1 or done % max(1, total // 10) == 0 or done == total:
                print(f"[AVERS] model=GBM | risk={risk_type} | a | t_idx={t_idx} | {100*done/total:6.2f}%")

        df = pd.DataFrame(rows)
        df.to_csv(f"sens_aversion_GBM_{risk_type}_a_t{t_idx}.csv", index=False)

# k: elw, entw
for risk_type in ["elw", "entw"]:
    base_kwargs = dict(risk_specs[risk_type])
    for t_idx in t_list:
        MR0, _ = sim.compute_MR(
            t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=base_kwargs,
            kind="Delta", max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose
        )

        rows = []
        total = len(k_grid)
        done = 0
        print(f"[AVERS] model=GBM | risk={risk_type} | param=k | t_idx={t_idx} | total={total}")

        for kv in k_grid:
            rk = dict(base_kwargs)
            rk["k"] = float(kv)

            MR, _ = sim.compute_MR(
                t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=rk,
                kind="Delta", max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose
            )

            rows.append({
                "model": "GBM", "risk_type": risk_type, "t_idx": float(t_idx),
                "param_name": "k", "param_value": float(kv),
                "MR": float(MR), "MR0": float(MR0), "DeltaMR": float(MR - MR0)
            })

            done += 1
            if done == 1 or done % max(1, total // 10) == 0 or done == total:
                print(f"[AVERS] model=GBM | risk={risk_type} | k | t_idx={t_idx} | {100*done/total:6.2f}%")

        df = pd.DataFrame(rows)
        df.to_csv(f"sens_aversion_GBM_{risk_type}_k_t{t_idx}.csv", index=False)

# beta: esl
risk_type = "esl"
base_kwargs = dict(risk_specs[risk_type])
for t_idx in t_list:
    MR0, _ = sim.compute_MR(
        t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=base_kwargs,
        kind="Delta", max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose
    )

    rows = []
    total = len(beta_list)
    done = 0
    print(f"[AVERS] model=GBM | risk=esl | param=beta | t_idx={t_idx} | total={total}")

    for bv in beta_list:
        rk = dict(base_kwargs)
        rk["beta"] = float(bv)

        MR, _ = sim.compute_MR(
            t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=rk,
            kind="Delta", max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose
        )

        rows.append({
            "model": "GBM", "risk_type": "esl", "t_idx": float(t_idx),
            "param_name": "beta", "param_value": float(bv),
            "MR": float(MR), "MR0": float(MR0), "DeltaMR": float(MR - MR0)
        })

        done += 1
        print(f"[AVERS] model=GBM | esl | beta | t_idx={t_idx} | {100*done/total:6.2f}%")

    df = pd.DataFrame(rows)
    df.to_csv(f"sens_aversion_GBM_esl_beta_t{t_idx}.csv", index=False)

# ============================================================
# Heston (kind="MinVar")
# ============================================================

sim = HestonSimulator(
    M=M, N=N, S0=S0, K=S0, T=T, t0=t0,
    mu=mu, sigma=sigma,
    kappa=kappa, theta=theta, sigma_v=sigma_v, corr=corr
)
sim.simulate_S()
sim.simulate_H()

# gamma: entl, ente, entw
for risk_type in ["entl", "ente", "entw"]:
    base_kwargs = dict(risk_specs[risk_type])
    for t_idx in t_list:
        MR0, _ = sim.compute_MR(
            t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=base_kwargs,
            kind="MinVar", max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose
        )

        rows = []
        total = len(gamma_grid)
        done = 0
        print(f"[AVERS] model=Heston | risk={risk_type} | param=gamma | t_idx={t_idx} | total={total}")

        for g in gamma_grid:
            rk = dict(base_kwargs)
            rk["gamma"] = float(g)

            MR, _ = sim.compute_MR(
                t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=rk,
                kind="MinVar", max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose
            )

            rows.append({
                "model": "Heston", "risk_type": risk_type, "t_idx": float(t_idx),
                "param_name": "gamma", "param_value": float(g),
                "MR": float(MR), "MR0": float(MR0), "DeltaMR": float(MR - MR0)
            })

            done += 1
            if done == 1 or done % max(1, total // 10) == 0 or done == total:
                print(f"[AVERS] model=Heston | risk={risk_type} | gamma | t_idx={t_idx} | {100*done/total:6.2f}%")

        df = pd.DataFrame(rows)
        df.to_csv(f"sens_aversion_Heston_{risk_type}_gamma_t{t_idx}.csv", index=False)

# a: ele, ente
for risk_type in ["ele", "ente"]:
    base_kwargs = dict(risk_specs[risk_type])
    for t_idx in t_list:
        MR0, _ = sim.compute_MR(
            t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=base_kwargs,
            kind="MinVar", max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose
        )

        rows = []
        total = len(a_grid)
        done = 0
        print(f"[AVERS] model=Heston | risk={risk_type} | param=a | t_idx={t_idx} | total={total}")

        for av in a_grid:
            rk = dict(base_kwargs)
            rk["a"] = float(av)

            MR, _ = sim.compute_MR(
                t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=rk,
                kind="MinVar", max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose
            )

            rows.append({
                "model": "Heston", "risk_type": risk_type, "t_idx": float(t_idx),
                "param_name": "a", "param_value": float(av),
                "MR": float(MR), "MR0": float(MR0), "DeltaMR": float(MR - MR0)
            })

            done += 1
            if done == 1 or done % max(1, total // 10) == 0 or done == total:
                print(f"[AVERS] model=Heston | risk={risk_type} | a | t_idx={t_idx} | {100*done/total:6.2f}%")

        df = pd.DataFrame(rows)
        df.to_csv(f"sens_aversion_Heston_{risk_type}_a_t{t_idx}.csv", index=False)

# k: elw, entw
for risk_type in ["elw", "entw"]:
    base_kwargs = dict(risk_specs[risk_type])
    for t_idx in t_list:
        MR0, _ = sim.compute_MR(
            t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=base_kwargs,
            kind="MinVar", max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose
        )

        rows = []
        total = len(k_grid)
        done = 0
        print(f"[AVERS] model=Heston | risk={risk_type} | param=k | t_idx={t_idx} | total={total}")

        for kv in k_grid:
            rk = dict(base_kwargs)
            rk["k"] = float(kv)

            MR, _ = sim.compute_MR(
                t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=rk,
                kind="MinVar", max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose
            )

            rows.append({
                "model": "Heston", "risk_type": risk_type, "t_idx": float(t_idx),
                "param_name": "k", "param_value": float(kv),
                "MR": float(MR), "MR0": float(MR0), "DeltaMR": float(MR - MR0)
            })

            done += 1
            if done == 1 or done % max(1, total // 10) == 0 or done == total:
                print(f"[AVERS] model=Heston | risk={risk_type} | k | t_idx={t_idx} | {100*done/total:6.2f}%")

        df = pd.DataFrame(rows)
        df.to_csv(f"sens_aversion_Heston_{risk_type}_k_t{t_idx}.csv", index=False)

# beta: esl
risk_type = "esl"
base_kwargs = dict(risk_specs[risk_type])
for t_idx in t_list:
    MR0, _ = sim.compute_MR(
        t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=base_kwargs,
        kind="MinVar", max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose
    )

    rows = []
    total = len(beta_list)
    done = 0
    print(f"[AVERS] model=Heston | risk=esl | param=beta | t_idx={t_idx} | total={total}")

    for bv in beta_list:
        rk = dict(base_kwargs)
        rk["beta"] = float(bv)

        MR, _ = sim.compute_MR(
            t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=rk,
            kind="MinVar", max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose
        )

        rows.append({
            "model": "Heston", "risk_type": "esl", "t_idx": float(t_idx),
            "param_name": "beta", "param_value": float(bv),
            "MR": float(MR), "MR0": float(MR0), "DeltaMR": float(MR - MR0)
        })

        done += 1
        print(f"[AVERS] model=Heston | esl | beta | t_idx={t_idx} | {100*done/total:6.2f}%")

    df = pd.DataFrame(rows)
    df.to_csv(f"sens_aversion_Heston_esl_beta_t{t_idx}.csv", index=False)

# ============================================================
# Jump (kind="MinVar")
# ============================================================

sim = JumpDiffusionSimulator(
    M=M, N=N, S0=S0, K=S0, T=T, t0=t0,
    mu=mu, sigma=sigma,
    lam=lam, meanJ=meanJ, stdJ=stdJ
)
sim.simulate_S()
sim.simulate_H()

# gamma: entl, ente, entw
for risk_type in ["entl", "ente", "entw"]:
    base_kwargs = dict(risk_specs[risk_type])
    for t_idx in t_list:
        MR0, _ = sim.compute_MR(
            t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=base_kwargs,
            kind="MinVar", max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose
        )

        rows = []
        total = len(gamma_grid)
        done = 0
        print(f"[AVERS] model=Jump | risk={risk_type} | param=gamma | t_idx={t_idx} | total={total}")

        for g in gamma_grid:
            rk = dict(base_kwargs)
            rk["gamma"] = float(g)

            MR, _ = sim.compute_MR(
                t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=rk,
                kind="MinVar", max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose
            )

            rows.append({
                "model": "Jump", "risk_type": risk_type, "t_idx": float(t_idx),
                "param_name": "gamma", "param_value": float(g),
                "MR": float(MR), "MR0": float(MR0), "DeltaMR": float(MR - MR0)
            })

            done += 1
            if done == 1 or done % max(1, total // 10) == 0 or done == total:
                print(f"[AVERS] model=Jump | risk={risk_type} | gamma | t_idx={t_idx} | {100*done/total:6.2f}%")

        df = pd.DataFrame(rows)
        df.to_csv(f"sens_aversion_Jump_{risk_type}_gamma_t{t_idx}.csv", index=False)

# a: ele, ente
for risk_type in ["ele", "ente"]:
    base_kwargs = dict(risk_specs[risk_type])
    for t_idx in t_list:
        MR0, _ = sim.compute_MR(
            t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=base_kwargs,
            kind="MinVar", max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose
        )

        rows = []
        total = len(a_grid)
        done = 0
        print(f"[AVERS] model=Jump | risk={risk_type} | param=a | t_idx={t_idx} | total={total}")

        for av in a_grid:
            rk = dict(base_kwargs)
            rk["a"] = float(av)

            MR, _ = sim.compute_MR(
                t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=rk,
                kind="MinVar", max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose
            )

            rows.append({
                "model": "Jump", "risk_type": risk_type, "t_idx": float(t_idx),
                "param_name": "a", "param_value": float(av),
                "MR": float(MR), "MR0": float(MR0), "DeltaMR": float(MR - MR0)
            })

            done += 1
            if done == 1 or done % max(1, total // 10) == 0 or done == total:
                print(f"[AVERS] model=Jump | risk={risk_type} | a | t_idx={t_idx} | {100*done/total:6.2f}%")

        df = pd.DataFrame(rows)
        df.to_csv(f"sens_aversion_Jump_{risk_type}_a_t{t_idx}.csv", index=False)

# k: elw, entw
for risk_type in ["elw", "entw"]:
    base_kwargs = dict(risk_specs[risk_type])
    for t_idx in t_list:
        MR0, _ = sim.compute_MR(
            t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=base_kwargs,
            kind="MinVar", max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose
        )

        rows = []
        total = len(k_grid)
        done = 0
        print(f"[AVERS] model=Jump | risk={risk_type} | param=k | t_idx={t_idx} | total={total}")

        for kv in k_grid:
            rk = dict(base_kwargs)
            rk["k"] = float(kv)

            MR, _ = sim.compute_MR(
                t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=rk,
                kind="MinVar", max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose
            )

            rows.append({
                "model": "Jump", "risk_type": risk_type, "t_idx": float(t_idx),
                "param_name": "k", "param_value": float(kv),
                "MR": float(MR), "MR0": float(MR0), "DeltaMR": float(MR - MR0)
            })

            done += 1
            if done == 1 or done % max(1, total // 10) == 0 or done == total:
                print(f"[AVERS] model=Jump | risk={risk_type} | k | t_idx={t_idx} | {100*done/total:6.2f}%")

        df = pd.DataFrame(rows)
        df.to_csv(f"sens_aversion_Jump_{risk_type}_k_t{t_idx}.csv", index=False)

# beta: esl
risk_type = "esl"
base_kwargs = dict(risk_specs[risk_type])
for t_idx in t_list:
    MR0, _ = sim.compute_MR(
        t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=base_kwargs,
        kind="MinVar", max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose
    )

    rows = []
    total = len(beta_list)
    done = 0
    print(f"[AVERS] model=Jump | risk=esl | param=beta | t_idx={t_idx} | total={total}")

    for bv in beta_list:
        rk = dict(base_kwargs)
        rk["beta"] = float(bv)

        MR, _ = sim.compute_MR(
            t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=rk,
            kind="MinVar", max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose
        )

        rows.append({
            "model": "Jump", "risk_type": "esl", "t_idx": float(t_idx),
            "param_name": "beta", "param_value": float(bv),
            "MR": float(MR), "MR0": float(MR0), "DeltaMR": float(MR - MR0)
        })

        done += 1
        print(f"[AVERS] model=Jump | esl | beta | t_idx={t_idx} | {100*done/total:6.2f}%")

    df = pd.DataFrame(rows)
    df.to_csv(f"sens_aversion_Jump_esl_beta_t{t_idx}.csv", index=False)


In [ ]:
# ============================================================
# GERAR TABELAS PARA K = 8, 10, 12 (3 strikes)
# - Produz 3 tabelas de Excel: K8, K10, K12
# - Dentro de cada tabela: primeiro bloco t=0.2 (GBM/Heston/Jump), depois t=0.8 (GBM/Heston/Jump)
# - Também gera 3 arquivos .tex (um por strike) com booktabs e ordem pedida
# ============================================================

# =========================
# CONFIG
# =========================
max_iter = 20
tol = 1e-4
alpha = 1e-3
verbose = False

t_list = [0.2, 0.8]
risk_order = ["ele", "elw", "entl", "ente", "entw", "esl"]
K_list = [8.0, 10.0, 12.0]   # 8 - 10 - 12

# métricas
metrics_t1 = [
    ("abs_acc_mean", "abs_acc"),
    ("abs_min", "abs_min"),
    ("abs_max", "abs_max"),
    ("rmse", "rmse"),
    ("q25", "q25"),
    ("q50", "q50"),
    ("q75", "q75"),
]
metrics_t2 = [
    ("MR_opt", "MR_opt"),
    ("MR_delta", "MR_delta"),
    ("improvement", "impr"),
    ("improvement_pct", "impr_%"),
]

# =========================
# Helpers internos (sem funções externas)
# =========================
def fmt_tex(x):
    if pd.isna(x):
        return ""
    return f"{x:.6g}"

def make_table_t_first(df, metrics):
    # retorna DataFrame index: "t=0.2 | GBM | abs_acc", columns=risk_order
    rows = []
    idx = []
    for t_idx in t_list:
        for model_name in ["GBM", "Heston", "Jump"]:
            for col_key, label in metrics:
                row = []
                for rt in risk_order:
                    v = df.loc[
                        (df["t_idx"] == float(t_idx)) &
                        (df["model"] == model_name) &
                        (df["risk_type"] == rt),
                        col_key
                    ].values
                    row.append(v[0] if len(v) else np.nan)
                rows.append(row)
                idx.append(f"t={t_idx:g} | {model_name} | {label}")
    tab = pd.DataFrame(rows, index=idx, columns=risk_order)
    return tab

def latex_table_by_t_then_model(tab, caption, label):
    # tab: index = "t=... | Model | metric", cols=risk_order
    colspec = "l" + "r"*len(risk_order)
    lines = []
    lines.append("\\begin{table}[!ht]")
    lines.append("\\centering")
    lines.append(f"\\caption{{{caption}}}")
    lines.append(f"\\label{{{label}}}")
    lines.append(f"\\begin{{tabular}}{{{colspec}}}")
    lines.append("\\toprule")
    lines.append(" & ".join([""] + risk_order) + " \\\\")
    lines.append("\\midrule")

    for t_idx in t_list:
        lines.append(f"\\multicolumn{{{1+len(risk_order)}}}{{l}}{{\\textbf{{$t={t_idx:g}$}}}} \\\\")
        lines.append("\\midrule")

        for model_name in ["GBM", "Heston", "Jump-Diffusion"]:
            key_model = "Jump" if model_name == "Jump-Diffusion" else model_name
            lines.append(f"\\multicolumn{{{1+len(risk_order)}}}{{l}}{{\\textbf{{{model_name}}}}} \\\\")
            lines.append("\\midrule")

            # filtra linhas desse (t, model)
            prefix = f"t={t_idx:g} | {key_model} | "
            sub = tab.loc[[i for i in tab.index if i.startswith(prefix)]]

            for ridx in sub.index:
                metric = ridx.split(" | ", 2)[2]
                row = [metric] + [fmt_tex(sub.loc[ridx, rt]) for rt in risk_order]
                lines.append(" & ".join(row) + " \\\\")
            lines.append("\\midrule")

    lines.append("\\bottomrule")
    lines.append("\\end{tabular}")
    lines.append("\\end{table}")
    return "\n".join(lines)

# ============================================================
# LOOP POR STRIKE: roda tudo e gera 3 tabelas
# ============================================================
for K_val in K_list:
    # =========================
    # 1) Instancia simuladores para esse strike
    # =========================
    models = [
        ("GBM",   GBMSimulator(M=M, N=N, S0=S0, K=float(K_val), T=T, t0=t0, mu=mu, sigma=sigma), "Delta"),
        ("Heston",HestonSimulator(M=M, N=N, S0=S0, K=float(K_val), T=T, t0=t0, mu=mu, sigma=sigma,
                                  kappa=kappa, theta=theta, sigma_v=sigma_v, corr=corr), "MinVar"),
        ("Jump",  JumpDiffusionSimulator(M=M, N=N, S0=S0, K=float(K_val), T=T, t0=t0, mu=mu, sigma=sigma,
                                         lam=lam, meanJ=meanJ, stdJ=stdJ), "MinVar"),
    ]

    # =========================
    # 2) Rodar e coletar
    # =========================
    records = []
    for model_name, sim, kind_opt in models:
        # garante paths e payoff/derivadas para o strike atual
        sim.simulate_S()
        sim.simulate_H()

        for rt in risk_order:
            rk = dict(risk_specs[rt])

            for t_idx in t_list:
                # hedge ótimo
                h_opt, _ = sim.optimize_hedge(
                    risk_type=rt,
                    risk_kwargs=rk,
                    t_idx=float(t_idx),
                    kind=kind_opt,
                    max_iter=max_iter,
                    tol=tol,
                    alpha=alpha,
                    verbose=verbose
                )

                # delta hedge
                h_delta = sim.init_control(kind="Delta")

                # -------- Tabela 1: métricas de |h_opt - h_delta| no trecho [t_idx, T] --------
                idx_start = int(np.floor((float(t_idx) - sim.t0) / sim.dt))
                idx_start = max(0, min(idx_start, sim.steps - 1))

                diff = np.asarray(h_opt) - np.asarray(h_delta)
                diff_seg = diff[:, idx_start:]
                absdiff = np.abs(diff_seg)

                abs_acc_time = absdiff.sum(axis=1)
                abs_acc_mean = float(abs_acc_time.mean())

                abs_min = float(absdiff.min()) if absdiff.size else np.nan
                abs_max = float(absdiff.max()) if absdiff.size else np.nan
                rmse = float(np.sqrt(np.mean(diff_seg**2))) if diff_seg.size else np.nan

                flat_abs = absdiff.reshape(-1)
                q25 = float(np.quantile(flat_abs, 0.25)) if flat_abs.size else np.nan
                q50 = float(np.quantile(flat_abs, 0.50)) if flat_abs.size else np.nan
                q75 = float(np.quantile(flat_abs, 0.75)) if flat_abs.size else np.nan

                # -------- Tabela 2: MR_opt vs MR_delta --------
                MR_opt, info_opt = sim.compute_MR(
                    t_idx=float(t_idx),
                    risk_type=rt,
                    risk_kwargs=rk,
                    kind=kind_opt,
                    max_iter=max_iter,
                    tol=tol,
                    alpha=alpha,
                    verbose=verbose
                )
                MR_del, info_del = sim.compute_MR(
                    t_idx=float(t_idx),
                    risk_type=rt,
                    risk_kwargs=rk,
                    kind="Delta",
                    max_iter=max_iter,
                    tol=tol,
                    alpha=alpha,
                    verbose=verbose
                )

                rho_T_opt = float(info_opt["rho_T"]) if isinstance(info_opt, dict) and "rho_T" in info_opt else np.nan
                rho_T_del = float(info_del["rho_T"]) if isinstance(info_del, dict) and "rho_T" in info_del else np.nan

                improvement = float(MR_del - MR_opt)
                improvement_pct = float(100.0 * improvement / abs(MR_del)) if MR_del != 0 else np.nan

                records.append({
                    "K": float(K_val),
                    "model": model_name,
                    "risk_type": rt,
                    "t_idx": float(t_idx),

                    "abs_acc_mean": abs_acc_mean,
                    "abs_min": abs_min,
                    "abs_max": abs_max,
                    "rmse": rmse,
                    "q25": q25,
                    "q50": q50,
                    "q75": q75,

                    "MR_opt": float(MR_opt),
                    "MR_delta": float(MR_del),
                    "rho_T_opt": rho_T_opt,
                    "rho_T_delta": rho_T_del,
                    "improvement": improvement,
                    "improvement_pct": improvement_pct,
                })

    df_all = pd.DataFrame(records)

    # =========================
    # 3) Montar tabelas (ordem: t=0.2 GBM/Heston/Jump, depois t=0.8 GBM/Heston/Jump)
    # =========================
    tab1 = make_table_t_first(df_all, metrics_t1)
    tab2 = make_table_t_first(df_all, metrics_t2)

    # =========================
    # 4) Salvar 3 tabelas (um arquivo por K, com 2 abas)
    # =========================
    xlsx_name = f"Tables_K{int(K_val)}.xlsx"
    with pd.ExcelWriter(xlsx_name, engine="openpyxl") as writer:
        tab1.to_excel(writer, sheet_name="Table1_hedge_diff")
        tab2.to_excel(writer, sheet_name="Table2_maturity_risk")

    # =========================
    # 5) Salvar LaTeX (um por K)
    # =========================
    tex1 = latex_table_by_t_then_model(
        tab1,
        caption=f"Optimal vs Delta hedge: summary statistics of $|h_{{opt}}-h_\\Delta|$ (K={K_val:g}).",
        label=f"tab:hedge_diff_K{int(K_val)}"
    )
    tex2 = latex_table_by_t_then_model(
        tab2,
        caption=f"Maturity risk comparison: optimized vs delta hedge (K={K_val:g}).",
        label=f"tab:maturity_risk_K{int(K_val)}"
    )

    tex_name = f"tables_K{int(K_val)}.tex"
    with open(tex_name, "w", encoding="utf-8") as f:
        f.write(tex1 + "\n\n" + tex2)

    print(f"Saved: {xlsx_name} and {tex_name}")

print("DONE. Generated 3 Excel files + 3 TeX files for K=8,10,12.")

